In [1]:
# Common imports
import numpy as np
import os
import pandas as pd



# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
from DataPipeline import DataFrameSelector
from DataPipeline import SetDummyVar
from sklearn.externals import joblib

__Läser in filen som ska köras__

In [4]:
df1 = pd.read_excel('Bergsåker20190223.xlsx')

In [5]:
df1 = df1.sort_values(['Lopp','Hast'], ascending = ['True','False']).reset_index()

In [7]:
# Importera model som ska användas
rf = joblib.load('Travmodel_v1.pkl')


In [8]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_v1.pkl')

__Definierar de variabler som ska användas i datasetet__

In [9]:
# Kategorivariabel
cat_attribs = ['GRUPP']
# Numerisk variabel 
num_attribs = ['V75PROC',
 'VLP',
 'SVLP',
 'VSVLP',
 'VPN_SUM',
 'VPN_SUM_ORD',
 'VPK_SUM',
 'VPK_SUM_ORD',
 'VLPB',
 'SVLPB',
 'VSVLPB',
 'E_P',
 'E_P_Num',
 'E_N',
 'E_R',
 'E_U',
 'G_R',
 'A_R',
 'T_R',
 'SP_R',
 'ToR',
 'P_R',
 'TK_R',
 'Ex_R',
 'R_R',
 'Ts_R']

__Kör modellen och plockar fram alla hästar över Score 0.089__

In [10]:

df1['GRUPP'] = df1.GRUPP.astype('object')
features = full_pipeline.fit_transform(df1)
predict = rf.predict_proba(features)
y_pred = np.where( predict[:,1] > 0.089,1,0)

In [11]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [12]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [13]:
# Gör om till lista och stoppar in i ett dictionary
pred = y_pred.tolist()
prediction = {'Predict':pred}
# Konverterar till Dataframe
df_pred = pd.DataFrame.from_dict(prediction)

In [14]:
df_predict1 = df1.merge(df_pred, right_index = True, left_index = True)
df_predict2 = df_predict1.merge(df_scored,  right_index = True, left_index = True) 

In [15]:
df1_out = df_predict2[df_predict2.Predict == 1][['Lopp','Hast','Score']]. \
sort_values(['Lopp','Score'], ascending = [True,False])
df1_out

,Lopp,Hast,Score
5,1,6,0.274980
1,1,2,0.134066
9,1,10,0.116039
0,1,1,0.091505
15,2,4,0.259572
23,2,12,0.165123
26,2,15,0.124386
19,2,8,0.090419
28,3,2,0.387461
27,3,1,0.169796


__Export till excel__

In [53]:
df1_out.to_excel('OuttakeBergsåker190223.xlsx', index = False)